# Third approach: templating
Here I try to populate a jsonld file, with a custom-made, predefined strcture, using templating from the original output file

In [2]:
import json
from pyld import jsonld
import utils
import requests

In [3]:
# with open('../examples_conductivity/single_result.json') as f:
#     data = json.load(f)

github_raw_url = "https://raw.githubusercontent.com/BIG-MAP/FINALES2_schemas/main/src/FINALES2_schemas/examples/results/conductivity/two_electrode.json?token=GHSAT0AAAAAACG24F2RRNP2CFIVIY4VEOYMZISZWNA"
response = requests.get(github_raw_url)
if response.status_code == 200:
    data = response.json()
else:
    print(response)
    data = {}


with open("./mapping3.json") as f:
    mapping = json.load(f)

<Response [404]>


In [4]:
data

{}

In [5]:
def process_data(data, mapping, view, last_mapped_key="root"):
    if isinstance(data, dict):
        for key, value in data.items():
            if key in mapping:
                print (key, "in mapping")
                last_mapped_key = key  # Update the last_mapped_key
                if isinstance(value, (dict, list)):
                    view[key] = process_data(value, mapping, {}, last_mapped_key)
                else:
                    view[key] = value
            elif last_mapped_key is not None:
                if isinstance(value, list):
                    view[last_mapped_key] = [process_data(item, mapping, {}, last_mapped_key) for item in value]
                elif isinstance(value, dict):
                    view[last_mapped_key] = process_data(value, mapping, {}, last_mapped_key)
    elif isinstance(data, list):
        view = []
        for item in data:
            if isinstance(item, (dict, list)):
                view.append(process_data(item, mapping, {}, last_mapped_key))
            elif isinstance(item, (int, float, str)):
                view.append(item)

    return view

view = process_data(data, mapping["@context"], {})
view

{}

In [6]:


def process_object(json_object, jsonld_dict:dict, counter=0):

    counter += 1
    print(f"#######   Recursion {counter} #######")
    print(json_object)

    if isinstance(json_object, list):

        for element in json_object:
            process_object(element, jsonld_dict, counter)

    elif isinstance(json_object, dict):

        for key, value in json_object.items():

            if key in mapping["@context"].keys():

                if isinstance(value,(str, int, float)):
        
                    jsonld_dict[key] = value

                elif isinstance(value, (dict, list)):

                    jsonld_dict[key] = {}

                    process_object(value, jsonld_dict, counter)



jsonld_dict = {}
process_object(data, jsonld_dict)
jsonld_dict

#######   Recursion 1 #######
{}


{}

In [7]:
def flatten_dict(d, parent_key='', separator='_'):
    items = {}
    for key, value in d.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            items.update(flatten_dict(value, new_key, separator=separator))
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, (dict, list)):
                    # Handle nested dictionaries and lists within lists
                    items.update(flatten_dict({str(i): item}, new_key, separator=separator))
                else:
                    items[f"{new_key}{separator}{i}"] = item
        else:
            items[new_key] = value
    return items

flattened_dict = flatten_dict(data, parent_key='', separator='/')
flattened_dict

{}

In [8]:
with open("./view_template.json") as f:
    template = json.load(f)

In [9]:
def get_value_from_nested_key(data:dict, path:list):
    current_data = data
    for key in path:
        if isinstance(current_data, dict) and key in current_data:
            current_data = current_data[key]
        else:
            return None  # Key path doesn't exist
    return current_data


def populate_template(data, template:dict, new_dict={}):

    for template_key, template_value in template.items(): #iterate over key and values

        if isinstance(template_value,str): #if value is a string

            if template_value.startswith("/"): # if string starts with "/", then its a path in original file               
                path_list = template_value.split("/")

                if "*" not in path_list: #if path does not have "*" then it points to a literal
                    new_dict[template_key] = get_value_from_nested_key(data, path_list)

                else:
                    pass
            else:
                new_dict[template_key] = template_value
        
        elif isinstance(template_value, dict):

            new_dict[template_key] = {}
            populate_template(template_value, template, new_dict=new_dict[template_key])


def process_template(template, original_dict):
    if isinstance(template, dict):
        result = {}
        for key, value in template.items():
            if isinstance(value, str):
                if value.startswith("/"):
                    result[key] = get_value_from_nested_key(original_dict, value.split("/"))
                else:
                    result[key] = value  # Keep non-path values as is
            elif isinstance(value, list):
                result[key] = []
                for item in value:
                    if isinstance(item, dict):
                        result[key].append(process_template(item, original_dict))
                    elif isinstance(item, str) and item.startswith("/"):
                        result[key].append(get_value_from_nested_key(original_dict, item.split("/")))
                    else:
                        result[key].append(item)  # Keep non-path values as is
            elif isinstance(value, dict):
                result[key] = process_template(value, original_dict)
            else:
                result[key] = value  # Keep other data types as is
        return result
    elif isinstance(template, list):
        result = []
        for item in template:
            if isinstance(item, dict):
                result.append(process_template(item, original_dict))
            elif isinstance(item, str) and item.startswith("/"):
                result.append(get_value_from_nested_key(original_dict, item.split("/")))
            else:
                result.append(item)  # Keep non-path values as is
        return result
    else:
        return template  # Keep other data types as is
        


In [ ]:
new_dict = process_template(data, template)
new_dict

: 